In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import *
import requests
from io import StringIO
from tqdm import tqdm

First get airport data by merging the top airports with the ICAO airport data

In [2]:
top = pd.read_csv('hw_5_data/top_airports.csv')
icao = pd.read_csv('hw_5_data/ICAO_airports.csv')
top_info = pd.merge(top, icao, left_on='ICAO', right_on='ident')
engine = create_engine('sqlite:///airports.db')
top_info.to_sql('airports', engine)

Create a new table in the database to hold all the weather data

In [3]:
metadata = MetaData()
weather_table = Table('weather', metadata,
                      Column('icao', String),
                      Column('date', String),
                      Column('min_temp', String),
                      Column('max_temp', String),
                      Column('mean_temp', String),
                      Column('min_humidity', String),
                      Column('max_humidity', String),
                      Column('mean_humidity', String),
                      Column('precip', String),
                      Column('event', String))
weather_table.create(bind=engine)

This is a webscraper function. Weather Underground provides csv files for up to a year of weather. To get 8 years of data, we need to loop over each year individually.

In [4]:
def get_weather(icao):
    base_url = 'https://www.wunderground.com/history/airport/'+icao.upper()
    df = None
    for year in range(2008, 2016):
        url = base_url+'/{0}/10/1/CustomHistory.html'.format(year)
        params = {'dayend': 1, 'monthend': 10, 'yearend': year+1, 'format': 1}
        r = requests.get(url, params=params)
        for line in StringIO(r.text).readlines()[3:]:
            data = line.split(',')[:-1]
            values = {'icao': icao,
                      'date': '{:02}-{:02}-{:02}'.format(*[int(x) for x in data[0].split('-')]), #format dates for SQL
                      'max_temp': data[1],
                      'mean_temp': data[2],
                      'min_temp': data[3],
                      'max_humidity': data[7],
                      'mean_humidity': data[8],
                      'min_humidity': data[9],
                      'precip': data[19],
                      'event': data[21]}
            engine.execute(weather_table.insert(), values)
    print(icao, 'done')

Now populate the database for all the airports in the airport table

In [5]:
icao_list = [r[0] for r in engine.execute("select ICAO from airports")]

# The function below will take quite some time to run

In [6]:
for icao in tqdm(icao_list):
    get_weather(icao)

KATL done
KORD done
KLAX done
KDFW done
KDEN done
KJFK done
KSFO done
KIAH done
KLAS done
KPHX done
KCLT done
KMIA done
KMCO done
KEWR done
KDTW done
KMSP done
KSEA done
KPHL done
KBOS done
KLGA done
KIAD done
KBWI done
KFLL done
KSLC done
PHNL done
KDCA done
KMDW done
KSAN done
KTPA done
KPDX done
KSTL done
KMCI done
KMEM done
KCLE done
KOAK done
TJSJ done
KRDU done
KBNA done
KSMF done
KHOU done
KSNA done
KAUS done
KSJC done
KMSY done
KPIT done
KSAT done
KCVG done
KMKE done
KDAL done
KIND done
